In [1]:
import requests
import pandas as pd
from datasets import load_dataset, DatasetDict, Dataset
from IPython.display import clear_output
import regex as re
import wikitextparser as wtp
import mwparserfromhell as mwp
import wikipedia as wiki

In [131]:
def parse_text(wikitext):
    text = mwp.parse(wikitext)
    text = text.strip_code()
    return text

In [50]:
# Made with ChatGPT, modified to use wikipedia package.
# Please note, due to the presence of disambiguation pages and empty articles there is no guarantee that num_articles
# pages will be returned. Instead, at most num_articles will be returned.
def mine_random_articles(num_articles):
    endpoint = "https://en.wikipedia.org/w/api.php"
    # Use UserAgent, gzip encoding in accordance with https://www.mediawiki.org/wiki/API:Etiquette
    headers = {'User-Agent': 'UofATextToTriplesDatasetCreation (tsadler@ualberta.ca)',
               "Accept-Encoding": "gzip"}
    params = {
        "format": "json",
        "action": "query",
        "generator": "random",
        "grnnamespace": 0,
        "prop": "",
        "grnlimit": num_articles
    }
    summaries = []
    response = requests.get(endpoint, params=params, headers=headers)
    data = response.json()

    random_articles = data["query"]["pages"]
    cnt = 0
    #for i in range(0, len(random_articles), 50):
    for article in random_articles:
        cnt += 1
        pageid = random_articles[article]["pageid"]
#         page_endpoint = "https://en.wikipedia.org/w/api.php"
#         page_params = {
#             "format": "json",
#             "action": "parse",
#             "prop": "text",
#             "pageid": ids
#         }
#         page_response = requests.get(page_endpoint, params=page_params)
#         print(page_response)
#         page_data = page_response.json()
#         page = page_data["parse"]
        # Ensure pages that simply redirect to other pages are not included in data.
        try:
            page = wiki.page(pageid=pageid)
        except wiki.DisambiguationError as e:
            continue
        if page.content.strip() != '':
            summaries.append([page.title, page.content, pageid])
        if cnt % 50 == 0:
            print(cnt)
    df = pd.DataFrame(summaries, columns=["title", "content", "pageid"])
    return df

In [51]:
# Maximum articles that can be mined at once seems to be 500.
data_new = mine_random_articles(500)
data_new

50
100
150
200
250
300
350
400
450
500


,title,content,pageid
0,Fédération Internationale des Sociétés Magiques,The International Federation of Magic Societie...,3725569
1,1940 Campeonato Carioca,The 1940 edition of the Campeonato Carioca kic...,61221370
2,Hired armed cutter Duke of Clarence,His Majesty's hired armed cutter Duke of Clare...,51314039
3,46–48 Brown Street,46–48 Brown Street is a grade-II building in M...,37119562
4,Arrondissement of Alençon,The arrondissement of Alençon is an arrondisse...,3314509
...,...,...,...
473,Manon Antoniazzi,"Manon Bonner Antoniazzi (née Jenkins, previou...",54863689
474,Ghairat Baheer,Ghairat Baheer is a citizen of Afghanistan who...,33231477
475,Albert Khinchagov,Albert Omarovich Khinchagov (Russian: Альберт ...,68577589
476,NTR: Netsuzou Trap,"NTR: Netsuzou Trap (Japanese: 捏造トラップ-NTR-, Hep...",51961711


In [52]:
print(data["content"][10])

2008 video game.mw-parser-output .infobox-subbox{padding:0;border:none;margin:-3px;width:auto;min-width:100%;font-size:100%;clear:none;float:none;background-color:transparent}.mw-parser-output .infobox-3cols-child{margin:auto}.mw-parser-output .infobox .navbar{font-size:100%}body.skin-minerva .mw-parser-output .infobox-header,body.skin-minerva .mw-parser-output .infobox-subheader,body.skin-minerva .mw-parser-output .infobox-above,body.skin-minerva .mw-parser-output .infobox-title,body.skin-minerva .mw-parser-output .infobox-image,body.skin-minerva .mw-parser-output .infobox-full-data,body.skin-minerva .mw-parser-output .infobox-below{text-align:center}.mw-parser-output .ib-video-game .infobox-label{white-space:nowrap;padding-right:0.65em}.mw-parser-output .ib-video-game .infobox-above{font-style:italic}Nancy Drew: The Haunting Of Castle MalloyDeveloper(s)Her InteractivePublisher(s)Her InteractivePlatform(s)Microsoft WindowsRelease.mw-parser-output .plainlist ol,.mw-parser-output .plain

In [53]:
data_new

,title,content,pageid
0,Fédération Internationale des Sociétés Magiques,The International Federation of Magic Societie...,3725569
1,1940 Campeonato Carioca,The 1940 edition of the Campeonato Carioca kic...,61221370
2,Hired armed cutter Duke of Clarence,His Majesty's hired armed cutter Duke of Clare...,51314039
3,46–48 Brown Street,46–48 Brown Street is a grade-II building in M...,37119562
4,Arrondissement of Alençon,The arrondissement of Alençon is an arrondisse...,3314509
...,...,...,...
473,Manon Antoniazzi,"Manon Bonner Antoniazzi (née Jenkins, previou...",54863689
474,Ghairat Baheer,Ghairat Baheer is a citizen of Afghanistan who...,33231477
475,Albert Khinchagov,Albert Omarovich Khinchagov (Russian: Альберт ...,68577589
476,NTR: Netsuzou Trap,"NTR: Netsuzou Trap (Japanese: 捏造トラップ-NTR-, Hep...",51961711


In [54]:
data = pd.concat([data, data_new])
data = data.reindex()

In [103]:
data = data.reset_index().drop(columns=['index'])
data

,title,content,pageid
0,Niyola,"Eniola Akinbo (Listen; born 9 December 1985), ...",43433888
1,Narayan nagbali,Narayan Nagbali is a three-day Hindu ritual pe...,4736808
2,Emerson Alcântara,Emerson Ricardo Alcântara (born 27 August 1970...,37125814
3,Red Scapular of the Passion,The Red Scapular of the Passion of Our Lord an...,21270348
4,Scott Walker: 30 Century Man,Scott Walker: 30 Century Man is a 2006 documen...,4636743
...,...,...,...
942,Manon Antoniazzi,"Manon Bonner Antoniazzi (née Jenkins, previou...",54863689
943,Ghairat Baheer,Ghairat Baheer is a citizen of Afghanistan who...,33231477
944,Albert Khinchagov,Albert Omarovich Khinchagov (Russian: Альберт ...,68577589
945,NTR: Netsuzou Trap,"NTR: Netsuzou Trap (Japanese: 捏造トラップ-NTR-, Hep...",51961711


Getting new content section for the data parsed with mwparserfromhell, as the wikipedia parsing is far better.

In [10]:
data['content']

'Abd al-Rahman al-Mahdi (983 – 4 March 1009), nicknamed Sanchol (\'little Sancho\', Sanchuelo to later historians), was the ʿĀmirid hajib (chief minister) of the Caliphate of Córdoba under Caliph Hisham II from October 1008, at a time when actual power in the caliphate was vested in the hajib. The Caliph nominated him as heir a month later, but he was deposed by a coup the following February. He was killed some weeks later during a vain attempt to regain power. Though an unpopular and highly flawed leader, his deposition led to the disintegration of the caliphate.\n\n\n== Youth ==\nSanchuelo was born in Córdoba, the son of the hajib Almanzor and a converted Christian named Abda. She was daughter of Sancho II of Pamplona, likely originally named Urraca or Sancha. He was nicknamed Sanchol (Arabic: شنجول, romanized: šanjūl), the diminutive of Sancho, after his Christian grandfather. His father had elevated the role of hajib, once an advisory role, to become the de facto leader of the Córd

In [93]:
disambig = []
for i in range(0, 500):
    try:
        page = wiki.page(pageid=data['pageid'][i])
        data['content'][i] = page.content
    except wiki.DisambiguationError as e:
        disambig.append(i)

C:\Users\tyms4\AppData\Local\Temp\ipykernel_5484\597531675.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['content'][i] = page.content
C:\Users\tyms4\anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\tyms4\anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [110]:
dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(data)
    }
)

In [111]:
dataset.push_to_hub("UofA-LINGO/wiki-articles", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
data = load_dataset('UofA-LINGO/wiki-articles')
data = pd.DataFrame(data['train'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/947 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/tyms4/.cache/huggingface/datasets/UofA-LINGO___parquet/UofA-LINGO--wiki-articles-63c6002f15ca2bb3/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
keywords = ['cities', 'companies', '']

In [88]:
#title = 'Big Ben'
#page = wiki.page(pageid=7290308)
page = wiki.page(title="List_of_cities_in_Alberta")
print(page.content)

PageError: Page id "list of cities in alberts" does not match any pages. Try another id!

In [79]:
page.categories

['Articles with short description',
 'Cities in North America',
 'Lists of cities by continent',
 'Lists of cities in the Americas',
 'North America-related lists',
 'Short description is different from Wikidata']

In [80]:
summary = page.summary
print(summary)

This is a list of cities in North America. For the most populous cities in North America, see List of North American cities by population.


In [77]:
summary = summary.replace('\n', '\n\n')
summary = re.sub(r'(?<=[a-zA-Z])\.(?=[a-zA-Z])', '.\n\n', summary)
print(summary)
pgraphs = summary.split('\n\n')
pgraphs

Big Ben is the nickname for the Great Bell of the Great Clock of Westminster, at the north end of the Palace of Westminster in London, England, and the name is frequently extended to refer also to the clock and the clock tower. The official name of the tower in which Big Ben is located was originally the Clock Tower, but it was renamed Elizabeth Tower in 2012 to mark the Diamond Jubilee of Elizabeth II.

The tower was designed by Augustus Pugin in a neo-Gothic style. When completed in 1859, its clock was the largest and most accurate four-faced striking and chiming clock in the world. The tower stands 316 feet (96 m) tall, and the climb from ground level to the belfry is 334 steps. Its base is square, measuring 40 feet (12 m) on each side. Dials of the clock are 22.5 feet (6.9 m) in diameter. All four nations of the UK are represented on the tower on shields featuring a rose for England, thistle for Scotland, shamrock for Ireland, and leek for Wales.

Big Ben is the largest of the towe

['Big Ben is the nickname for the Great Bell of the Great Clock of Westminster, at the north end of the Palace of Westminster in London, England, and the name is frequently extended to refer also to the clock and the clock tower. The official name of the tower in which Big Ben is located was originally the Clock Tower, but it was renamed Elizabeth Tower in 2012 to mark the Diamond Jubilee of Elizabeth II.',
 'The tower was designed by Augustus Pugin in a neo-Gothic style. When completed in 1859, its clock was the largest and most accurate four-faced striking and chiming clock in the world. The tower stands 316 feet (96 m) tall, and the climb from ground level to the belfry is 334 steps. Its base is square, measuring 40 feet (12 m) on each side. Dials of the clock are 22.5 feet (6.9 m) in diameter. All four nations of the UK are represented on the tower on shields featuring a rose for England, thistle for Scotland, shamrock for Ireland, and leek for Wales.',
 "Big Ben is the largest of 

In [83]:
sections = re.findall(r'= (.*?) =', page.content)
sections

['Antigua and Barbuda',
 'Aruba',
 'Bahamas',
 'Barbados',
 'Belize',
 'Bermuda',
 'British Virgin Islands',
 'Canada',
 'Cayman Islands',
 'Costa Rica',
 'Cuba',
 'Dominica',
 'Dominican Republic',
 'El Salvador',
 'Greenland',
 'Grenada',
 'Guatemala',
 'Haiti',
 'Honduras',
 'Jamaica',
 'Mexico',
 'Montserrat',
 'Netherlands Antilles',
 'Nicaragua',
 'Panama',
 'Puerto Rico',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint-Pierre and Miquelon',
 'Saint Vincent and the Grenadines',
 'Trinidad and Tobago',
 'Turks and Caicos Islands',
 'United States',
 'United States Virgin Islands',
 'See also',
 'References']

In [84]:
for section in sections:
    if page.section(section) != '':
        print(page.section(section))
        print()

Oranjestad

Nassau
Freeport City

Bridgetown
Speightstown

Belize City
Belmopan

Toronto
Montreal
Vancouver
Ottawa
Calgary
Edmonton
Winnipeg
Halifax

George Town

Havana
Santiago de Cuba

Roseau

Santo Domingo
Santiago de los Caballeros
San Felipe de Puerto Plata

San Salvador
San Miguel
Santa Ana

Nuuk
Sisimiut
Ilulissat
Qaqortoq
Aasiaat

Guatemala City

Port-au-Prince
Delmas
Pétion-Ville
Cap-Haïtien
Gonaïves
Port-de-Paix
Jacmel
Les Cayes
Jérémie

Tegucigalpa
San Pedro Sula
Choloma
La Ceiba
El Progreso
Choluteca
Comayagua
Puerto Cortés
La Lima
Danlí

Kingston
Montego Bay
Spanish Town
Portmore
Ocho Rios

Mexico City
Guadalajara
Monterrey
Puebla
Toluca
Tijuana
Chihuahua
Juárez
León
Torreón
San Luis Potosí
Mérida
Acapulco
Cancún
Los Cabos
Cuernavaca
Pachuca
Querétaro

Managua
Leon

San Juan

Castries

Port of Spain
San Fernando
Chaguanas
Arima
Couva
Point Fortin
Scarborough
Tunapuna
Diego Martin
Princes Town
Penal
Siparia
San Juan
Sangre Grande
Mayaro

Cockburn Town

Boston, Massachusett